<a href="https://colab.research.google.com/github/binny-mathew/IITKGP_CS69002_Spring_2019/blob/master/Sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movie Review Sentiment Analysis

## Import Header files

In [0]:
import torch
import pandas as pd
import numpy as np

In [4]:
print('Welcome to Computing Lab 2. The current PyTorch version is ', torch.__version__)

Welcome to Computing Lab 2. The current PyTorch version is  1.0.1.post2


## Load the dataset and visualize


There are 3 ways to do this:

(a)  From Github (Files < 25MB)

The easiest way to upload a CSV file is from your GitHub repository. Click on the dataset in your repository, then click on View Raw. Copy the link to the raw dataset and store it as a string variable called url in Colab as shown below (a cleaner method but it’s not necessary). The last step is to load the url into Pandas read_csv to get the dataframe.


```
url = 'copied_raw_GH_link'
df1 = pd.read_csv(url)
# Dataset is now stored in a Pandas Dataframe

```

---


(b) From a local drive

To upload from your local drive, start with the following code:

```
from google.colab import files
uploaded = files.upload()
```

It will prompt you to select a file. Click on “Choose Files” then select and upload the file. Wait for the file to be 100% uploaded. You should see the name of the file once Colab has uploaded it.

Finally, type in the following code to import it into a dataframe (make sure the filename matches the name of the uploaded file).
```
import io
df2 = pd.read_csv(io.BytesIO(uploaded['Filename.csv']))
# Dataset is now stored in a Pandas Dataframe
```

---


(c) From Google drive

This the most complicated of the three methods. Follow this [link](https://medium.freecodecamp.org/how-to-transfer-large-files-to-google-colab-and-remote-jupyter-notebooks-26ca252892fa) to learn more.

In [9]:
from google.colab import files
uploaded = files.upload()

Saving check.csv to check (1).csv


In [12]:
type(uploaded), uploaded.keys(), type(uploaded['check.csv'])

(dict, dict_keys(['check.csv']), bytes)

In [46]:
import io
df = pd.read_csv(io.StringIO(uploaded['check.csv'].decode('utf-8')), sep='\t')
df.columns

Index(['text', 'label'], dtype='object')

In [47]:
df['text'] = df['text'].astype('str')
df.dtypes

text     object
label     int64
dtype: object

## Pandas

[Pandas](https://pandas.pydata.org/) is a high-performance, easy-to-use data structure and data analytics tool for Python.

Tutorials:

*   [Link 1](https://pandas.pydata.org/pandas-docs/stable/getting_started/tutorials.html)
*   [Link 2](https://www.tutorialspoint.com/python_pandas)
*   [Link 3](https://www.dataquest.io/blog/pandas-python-tutorial/)



In [29]:
print('Number of Negative movie reviews', len(df[df['label']==0]))
print('Number of Positive movie reviews', len(df[df['label']==1]))

Number of Negative movie reviews 246
Number of Positive movie reviews 253


## Data pre-processing
The first step when building a neural network model is getting your data into the proper form to feed into the network. We'll need to encode each word with an integer. We'll also want to clean it up a bit.

You can use any type of pre-processing that you see fit, based on the task. For the tutorial, i will show just one pre-processing step: **lowercasing**.

In [39]:
df['text1'] = map(lambda x: x.lower(), df['text'].str)
df['text1'] = df['text1'].astype('str') 
df

,text,label,text1
0,John Waters has given us a genuinely enjoyable...,1,<map object at 0x7fecfca11a90>
1,This first two seasons of this comedy series w...,1,<map object at 0x7fecfca11a90>
2,"What an unfortunate mess is ""Shiner."" I wanted...",0,<map object at 0x7fecfca11a90>
3,I'm not entirely sure Rob Schmidt qualifies as...,1,<map object at 0x7fecfca11a90>
4,i wasn't sure whether to laugh or cry. Porrett...,0,<map object at 0x7fecfca11a90>
5,Saw this movie twice at community screenings a...,1,<map object at 0x7fecfca11a90>
6,A friend once asked me to read a screenplay of...,0,<map object at 0x7fecfca11a90>
7,........and an extremely bad one at that!!! Ho...,0,<map object at 0x7fecfca11a90>
8,I've read a lot of comments about the film and...,1,<map object at 0x7fecfca11a90>
9,After having seen the movie the first question...,0,<map object at 0x7fecfca11a90>


In [42]:
df.columns

Index(['text', 'label'], dtype='object')